In [ ]:
# Basic libraries
import time
import argparse

from os.path import abspath
from os.path import dirname as up
import numpy as np
import sys
import h5py
import math

# Insert path to pybf library to system path
print(up(up(up(abspath("__file__")))))
path_to_lib = up(up(up(abspath("__file__"))))
print(path_to_lib)
sys.path.insert(0, path_to_lib)

#Import libraries and functions
from pybf.pybf.io_interfaces import DataLoader
from pybf.pybf.signal_processing import demodulate_decimate
from pybf.pybf.signal_processing import interpolate_modulate
from pybf.pybf.signal_processing import filter_band_pass
from pybf.scripts.beamformer_cartesian_realtime import BFCartesianRealTime
from pybf.pybf.transducer import Transducer
from pybf.pybf.image_settings import ImageSettings
from pybf.pybf.visualization import plot_image
from pybf.scripts.beamformer_cartesian import beamformer_cartesian

In [ ]:
# Beamform and plot the original image

dataset_path = "../datasets/archive/archive_to_download/database/experiments/resolution_distorsion/rf_dataset.hdf5"

data_loader_obj = DataLoader(dataset_path)

### Specify Image settings and create corresponding object ###

img_res = [400, 600]
image_x_range = [-0.019, 0.019]
image_z_range = [0.005, 0.05]

db_range = 50

LATERAL_PIXEL_DENSITY_DEFAULT = 5

img_config = ImageSettings(image_x_range[0],
                           image_x_range[1],
                           image_z_range[0],
                           image_z_range[1],
                           LATERAL_PIXEL_DENSITY_DEFAULT,
                           data_loader_obj.transducer)

### Specify preprocessing parameters for RF data ###

decimation_factor = 1
interpolation_factor = 10

### Specify TX strategy and Apodization parameters ###

start_time = 0
correction_time_shift = 0

alpha_fov_apod = 40

### Specify Sampling Frequency ###

SAMPLING_FREQ = 20.832 * (10 ** 6)

filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                         data_loader_obj.tx_strategy,
                         data_loader_obj.transducer,
                         decimation_factor,
                         interpolation_factor,
                         img_res,
                         img_config,
                         start_time=start_time,
                         correction_time_shift=correction_time_shift,
                         alpha_fov_apod=alpha_fov_apod,
                         bp_filter_params=filters_params,
                         envelope_detector='hilbert',
                         picmus_dataset=True)

rf_data_shape = (data_loader_obj.num_of_acq_per_frame,) + data_loader_obj.get_rf_data(0, 0).shape
print('RF data shape: ', rf_data_shape)

rf_data = np.zeros(rf_data_shape)
for i in range(rf_data.shape[0]):
    rf_data[i, :, :] = data_loader_obj.get_rf_data(0, i)

# Beamforming
img_data = bf.beamform(rf_data, numba_active=True)

_ = plot_image(np.abs(img_data), 
               scatters_coords_xz=None,
               elements_coords_xz=None,
               framework='plotly',
               title='Original Image (PW_75)',
               image_x_range=image_x_range,
               image_z_range=image_z_range,
               db_range=db_range,
               colorscale='Greys',
               save_fig=True, 
               show=True,
               path_to_save='.')

In [ ]:
def get_original_rf_data(path_to_dataset):
    
    # Read file
    data_loader_obj = DataLoader(path_to_dataset)
    
    rf_data_shape = (data_loader_obj.num_of_acq_per_frame,) + data_loader_obj.get_rf_data(0, 0).shape
    print('RF data shape: ', rf_data_shape)

    rf_data = np.zeros(rf_data_shape)
    for i in range(rf_data.shape[0]):
        rf_data[i, :, :] = data_loader_obj.get_rf_data(0, i)
        
    return rf_data

In [ ]:
rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/contrast_speckle/rf_dataset.hdf5")

print(rf_original_data[0][1][223].dtype)

In [ ]:
print(np.max(rf_original_data[:][:]))

print(np.min(rf_original_data[:][:]))

In [ ]:
# Pass all the values of the dataset through the function and quantize them into integers between 0 and 4095

rf_original_data = get_original_rf_data("../datasets/archive/archive_to_download/database/experiments/resolution_distorsion/rf_dataset.hdf5")

# Adjust range of original data to positive values
rf_range_data = rf_original_data + 1
print(rf_range_data.shape)

bits_ADC = 10
full_range = 2

# Multiply all values with the following
mult_with = ((2**bits_ADC)-1)/full_range

mult_data = rf_range_data*mult_with
print(mult_data.shape)
flatten_mult_data = mult_data.flatten()
print(flatten_mult_data.shape)

# Convert to integers
int_converted_data = np.zeros(rf_original_data.shape, dtype='int32')
flatten_conv_data = int_converted_data.flatten()
print(flatten_conv_data.shape)

i = 0
for sample in flatten_mult_data:
    flatten_conv_data[i] = int(np.round(sample))
    i = i+1
    
int_converted_data = flatten_conv_data.reshape(rf_original_data.shape)
print(int_converted_data.shape)
    
print("Done.")


In [ ]:
# Test analogue values with their integer conversion outputs
print(rf_range_data[0][2][2])
print(int_converted_data[0][2][2])

In [ ]:
# Write the integer output values to a file
with open('./BL_Coding/contrast_speckle/int_converted_values.npy', 'wb') as f:
    np.save(f, int_converted_data)
    
f.close()

In [ ]:
# Load integer conversion output values from file
import numpy as np
with open('./BL_Coding/contrast_speckle/int_converted_values.npy', 'rb') as f:
    int_out_values = np.load(f)
f.close()
print(int_out_values.shape)
#print(int_converted_data[2][2][3])
print(int_out_values[2][2][3])

In [ ]:
# Plot signals before and after conversion to make sure they look similar
# Plot traces from a specific acquisition and channel
from pybf.pybf.visualization import plot_trace

# Original Traces
path_to_save_original = "./Original_Traces"
_ = plot_trace(rf_original_data, channel = 1, framework='plotly', save_fig=False, show=True, path_to_save=path_to_save_original)

# ADC output Traces
path_to_save_int = "./Int_Conversion_Output_Traces"
_ = plot_trace(int_out_values, channel = 1, framework='plotly', save_fig=False, show=True, path_to_save=path_to_save_int)

In [ ]:
# Convert integers to BL encoding values
import numpy as np
import math
import json

# CREATE CLASS WITH MULTIPLE ARRAYS
# ARRAY TO STORE PRE-FIXES
# ARRAY TO STORE SUFFIXES
# ARRAY TO STORE CONCATENATED VERSIONS
# ARRAY TO STORE LENGTH OF PRE-FIX
# ARRAY TO STORE LENGTH OF SUFFIX

class BL_encoder():
    
    def __init__(self,
                 factor_S = 1):
        
        self._factor_S = factor_S
        
    def convert_to_int(self, path_to_dataset, path_to_save, bits_ADC, full_range):
        
        # Pass all the values of the dataset through the function and quantize them into integers between 0 and 4095
        # Read file
        data_loader_obj = DataLoader(path_to_dataset)

        rf_data_shape = (data_loader_obj.num_of_acq_per_frame,) + data_loader_obj.get_rf_data(0, 0).shape
        print('RF data shape: ', rf_data_shape)

        rf_data = np.zeros(rf_data_shape)
        for i in range(rf_data.shape[0]):
            rf_data[i, :, :] = data_loader_obj.get_rf_data(0, i)
            
        rf_original_data = rf_data
        print("RF Original Data shape: {}".format(rf_original_data.shape))
        
        # Adjust range of original data to positive values
        rf_range_data = rf_original_data + 1

        bits_ADC = bits_ADC
        full_range = full_range

        # Multiply all values with the following
        mult_with = ((2**bits_ADC)-1)/full_range

        mult_data = rf_range_data*mult_with
        print(mult_data.shape)
        flatten_mult_data = mult_data.flatten()
        print(flatten_mult_data.shape)

        # Convert to integers
        int_converted_data = np.zeros(rf_original_data.shape, dtype='int32')
        flatten_conv_data = int_converted_data.flatten()
        print(flatten_conv_data.shape)

        i = 0
        for sample in flatten_mult_data:
            flatten_conv_data[i] = int(np.round(sample))
            i = i+1

        int_converted_data = flatten_conv_data.reshape(rf_original_data.shape)
        print("Converted data shape: {}".format(int_converted_data.shape))

        print("Done converting to integers.")
        
        # Write the integer output values to a file
        with open(path_to_save, 'wb') as f:
            np.save(f, int_converted_data)

        f.close()
        
        # Load integer conversion output values from file
        with open(path_to_save, 'rb') as f:
            int_out_values = np.load(f)
            
        f.close()
        print("Recovered ints data shape: {}".format(int_out_values.shape))
        
        return int_converted_data

        
        
    def encode_data(self, path_to_int_array):
        
        S = self._factor_S
        
        # Load integer samples
        with open(path_to_int_array, 'rb') as f:
            int_array = np.load(f)
            
        f.close()
        
        # Calculate M
        M = np.ceil(np.log2((int_array+(2**S))/(2**S)))
        M = M.astype('int32')
        print("Calculated M.")
        
        # Calculate K
        y = (1 + np.sqrt(1 + 8*M))/2
        K = np.ceil(y)-1
        K = K.astype('int32')
        print("Calculated K.")
        
        # Calculate X
        X = M - (K*(K-1)/2)
        X = X.astype('int32')
        print("Calculated X.")
        
        # Calculate number of zeros in prefix
        numb_zeros = K - (X-1)
        flatten_numb_zeros = numb_zeros.flatten()

        zeros = []
        i = 0
        for sample in flatten_numb_zeros:
            zeros.append(format(0, '#0{}b'.format(sample+2)))
            i = i+1
            
        print("Calculated number of zeros in prefix.")
            
        # Calculate number of ones in prefix
        numb_ones = X-1
        max_val = (2**numb_ones)-1
        flatten_numb_ones = numb_ones.flatten()
        flatten_max_val = max_val.flatten()

        ones = []
        for k in range(flatten_numb_ones.shape[0]):
            max_value = flatten_max_val[k]
            ones.append(format(max_value, '#0{}b'.format(flatten_numb_ones[k]+2)))
            
        print("Calculated number of ones in prefix.")
        
        # Calculate binary clusters
        binary_cluster = []


        for i in range(flatten_numb_zeros.shape[0]):
            single_bl = '0b1'

            if (flatten_numb_zeros[i] != 0):
                for char in zeros[i][2:]:
                    single_bl = single_bl + char
            if (flatten_numb_ones[i] != 0):
                for char in ones[i][2:]:
                    single_bl = single_bl + char

            binary_cluster.append(single_bl)
            
        print("Calculated binary clusters.")
            
        # Calculate prefixes
        prefix = []

        for i in range(len(binary_cluster)):
            single_pr = '0b'
            stringlength = len(binary_cluster[i])
            string_sliced = binary_cluster[i][stringlength::-1]

            for char in string_sliced[:-2]:
                # Reverse number
                single_pr = single_pr + char

            prefix.append(single_pr)
            
        print("Calculated prefixes.")
            
        # Calculate suffixes and lengths of suffixes
        suffix_dec = int_array - ((2**S)*(2**(M-1)-1)) - 1
        suffix_dec = suffix_dec.astype('int32')
        
        suffix_length = M + (S-1)
        suffix_length = suffix_length.astype('int32')
        
        suffix = []

        flatten_suffix_dec = suffix_dec.flatten()
        flatten_suffix_length = suffix_length.flatten()

        for k in range(flatten_suffix_dec.shape[0]):

            if (flatten_suffix_length[k] != 0):
                suffix.append(format(flatten_suffix_dec[k], '#0{}b'.format(flatten_suffix_length[k]+2)))
        
        print("Calculated suffixes and lengths of suffixes.")
        
        # Concatenate prefixes and suffixes to get the BL codes and their lengths
        bl_codes = []
        bl_codes_length = []

        for i in range(len(suffix)):
            single_bl_code = '0b'

            for char in prefix[i][2:]:
                single_bl_code = single_bl_code + char
            if (flatten_suffix_length[i] != 0):
                for char in suffix[i][2:]:
                    single_bl_code = single_bl_code + char

            bl_codes.append(single_bl_code)
            bl_codes_length.append(len(single_bl_code[2:]))
            
        # Return lists of bl_codes and bl_codes_length
        print("Shape of BL codes array: {}".format(len(bl_codes)))
        print("Shape of BL lengths array: {}".format(len(bl_codes_length)))
        print("Done.")
        
        return bl_codes, bl_codes_length
    
    
    # Save codes and lengths in a file
    def save_data(self, path_to_save, name_codes, bl_codes, name_lengths, bl_codes_length):

        # Write the BL codes and lengths to a file
        with open(path_to_save + name_codes + '.npy', 'w') as f:
            json.dump(bl_codes, f)

        f.close()

        with open(path_to_save + name_lengths + '.npy', 'w') as f:
            json.dump(bl_codes_length, f)

        f.close()
        
    # Load BL codes and lengths from files
    def load_data(self, path_to_bl_codes_data, path_to_bl_lengths_data):
        
        with open(path_to_bl_codes_data, 'r') as f:
            bl_codes = json.load(f)

        f.close()

        with open(path_to_bl_lengths_data, 'r') as f:
            bl_codes_length = json.load(f)

        f.close()
        
        return bl_codes, bl_codes_length
    
    def compression_ratio(self, integer_array, bl_codes_length):
        
        # Calculate compression ratio

        # Initial size
        initial_number_bits = (integer_array.size)*12
        print("Initial size: {} bits.".format(initial_number_bits))

        # Compressed number of bits
        compressed_number_bits = np.sum(bl_codes_length, dtype='int')
        print("Compressed size: {} bits".format(compressed_number_bits))

        # Compression ratio
        compr_ratio = (compressed_number_bits/initial_number_bits)*100
        print("The compressed data is {:.2f}% of the original data.".format(compr_ratio))
        
        return compr_ratio
        
        

In [ ]:
# Declare the encoder
encoder = BL_encoder()

# Convert the dataset from floats to integers in the range of the ADC
integer_array = encoder.convert_to_int("../datasets/archive/archive_to_download/database/experiments/contrast_speckle/rf_dataset.hdf5", "./BL_Coding/int_converted_values.npy", 12, 2)


In [ ]:
# Encode data

encoder = BL_encoder()
bl_codes, bl_codes_length = encoder.encode_data("./BL_Coding/int_converted_values.npy")

In [ ]:
# Save BL codes and their lengths to a file
encoder = BL_encoder()
encoder.save_data("./BL_Coding/", "BL_Codes_Contrast_Speckle_RF", bl_codes, "BL_Lengths_Contrast_Speckle_RF", bl_codes_length)

# Load BL codes and their lengths from a file
bl_codes_file, bl_codes_length_file = encoder.load_data("./BL_Coding/BL_Codes_Contrast_Speckle_RF.npy", "./BL_Coding/BL_Lengths_Contrast_Speckle_RF.npy")

print(len(bl_codes_file))
print(len(bl_codes_length_file))

In [ ]:
# Load integer conversion output values from file
import numpy as np
with open('./BL_Coding/int_converted_values.npy', 'rb') as f:
    integer_array = np.load(f)
f.close()
print(integer_array.shape)
#print(integer_array[2][2][3])
print(integer_array[2][2][3])

In [ ]:
# Calculate compression ratio

# Initial size
initial_number_bits = (integer_array.size)*12
print("Initial size: {} bits.".format(initial_number_bits))

# Compressed number of bits
compressed_number_bits = np.sum(bl_codes_length_file, dtype='int')
print("Compressed size: {} bits".format(compressed_number_bits))

# Compression ratio
compr_ratio = (compressed_number_bits/initial_number_bits)*100
print("The compressed data is {}% of the original data.".format(compr_ratio))


In [ ]:
compression = encoder.compression_ratio(integer_array, bl_codes_length)

In [ ]:
# Decode values and get the integers back

class BL_decoder():
    
    def __init__(self,
                 factor_S = 1):
        
        self._factor_S = factor_S

    
    def decode_data(self, path_to_bl_codes_data):
        
        S = self._factor_S
        
        with open(path_to_bl_codes_data, 'r') as f:
            bl_codes = json.load(f)

        f.close()
        
        print("Shape of BL codes: {}".format(len(bl_codes)))
    
        decoded_integers = np.zeros((len(bl_codes),), dtype=np.int32)

        for i in range(len(bl_codes)):

            for bit in range(2, len(bl_codes[i])-1):

                if (bl_codes[i][bit] == '0' and bl_codes[i][bit+1] == '1'):
                    prefix_dec = bl_codes[i][2:bit+2]
                    break

            prefix_length = len(prefix_dec)

            K = prefix_length - 1

            T = 0

            if (prefix_dec[0] == '1'):
                T = 1
                for k in range(len(prefix_dec)):
                    if (prefix_dec[k] == '1' and prefix_dec[k+1] == '1'):
                        T = T + 1
                    elif (prefix_dec[k+1] == '0'):
                        break
                        
            M = int(((K*(K-1))/2) + T + 1)

            suffix_length = M + (S-1)

            suffix = bl_codes[i][2+prefix_length:(2+prefix_length) + suffix_length]

            suffix_dec = int(suffix,2)

            Z = suffix_dec + ((2**S)*((2**(M-1))-1)) + 1

            decoded_integers[i] = Z
            
            sys.stdout.write('\r')
            sys.stdout.write("{} out of 31948800".format(i))
            
            
        reshaped_decoded_int = np.reshape(decoded_integers, (75,128,3328))  
        
        print("Done.")
        
        return reshaped_decoded_int
    
    
    def save_decoded_int(self, name_file, reshaped_decoded_int):
        
        # Save decoded integers array into file
        with open(name_file + '.npy', 'wb') as f:
            np.save(f, reshaped_decoded_int)

        f.close()
        
        
    def load_decoded_int(self, name_file):
        
        # Read file with decoded integers
        with open(name_file, 'rb') as f:
            reshaped_decod_int_file = np.load(f)

        f.close()
        
        return reshaped_decod_int_file
        
        

In [ ]:
def do_everything(type_data,dataset):
    
    if (type_data == 'RF'):
        typed = 'rf'
        
    elif (type_data == 'IQ'):
        typed = 'iq'

    
    if dataset == 0:
        path = "resolution_distorsion"
        path_to_dataset = "../datasets/archive/archive_to_download/database/experiments/resolution_distorsion/" + typed + "_dataset.hdf5"
    elif dataset == 1:
        path = "contrast_speckle"
        path_to_dataset = "../datasets/archive/archive_to_download/database/experiments/contrast_speckle/" + typed + "_dataset.hdf5"

        
    # Beamform and plot the original image    
    data_loader_obj = DataLoader(path_to_dataset)
    
    
    ### Specify Image settings and create corresponding object ###

    img_res = [400, 600]
    image_x_range = [-0.019, 0.019]
    image_z_range = [0.005, 0.05]

    db_range = 50

    LATERAL_PIXEL_DENSITY_DEFAULT = 5

    img_config = ImageSettings(image_x_range[0],
                               image_x_range[1],
                               image_z_range[0],
                               image_z_range[1],
                               LATERAL_PIXEL_DENSITY_DEFAULT,
                               data_loader_obj.transducer)

    ### Specify preprocessing parameters for RF data ###

    decimation_factor = 1
    interpolation_factor = 10

    ### Specify TX strategy and Apodization parameters ###

    start_time = 0
    correction_time_shift = 0

    alpha_fov_apod = 40

    ### Specify Sampling Frequency ###

    SAMPLING_FREQ = 20.832 * (10 ** 6)

    filters_params = [1 * 10 **6, 8 * 10 **6, 0.5 * 10 **6]

    bf = BFCartesianRealTime(data_loader_obj.f_sampling,
                             data_loader_obj.tx_strategy,
                             data_loader_obj.transducer,
                             decimation_factor,
                             interpolation_factor,
                             img_res,
                             img_config,
                             start_time=start_time,
                             correction_time_shift=correction_time_shift,
                             alpha_fov_apod=alpha_fov_apod,
                             bp_filter_params=filters_params,
                             envelope_detector='hilbert',
                             picmus_dataset=True)

    rf_data_shape = (data_loader_obj.num_of_acq_per_frame,) + data_loader_obj.get_rf_data(0, 0).shape
    print(typed + ' data shape: ', rf_data_shape)

    rf_data = np.zeros(rf_data_shape)
    for i in range(rf_data.shape[0]):
        rf_data[i, :, :] = data_loader_obj.get_rf_data(0, i)

    # Beamforming
    img_data = bf.beamform(rf_data, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title='Original Image (PW_75)',
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=True, 
                   show=True,
                   path_to_save='.')
    
    # Declare the encoder
    encoder = BL_encoder()

    # Convert the dataset from floats to integers in the range of the ADC
    integer_array = encoder.convert_to_int(path_to_dataset, "./BL_Coding/" + path + "/int_converted_values.npy", 12, 2)
    
    # Encode data
    bl_codes, bl_codes_length = encoder.encode_data("./BL_Coding/" + path + "/int_converted_values.npy")
    
    # Save BL codes and their lengths to a file
    encoder = BL_encoder()
    encoder.save_data("./BL_Coding/", path + "/BL_Codes_" + path, bl_codes, path + "/BL_Lengths_" + path, bl_codes_length)

    # Load BL codes and their lengths from a file
    bl_codes_file, bl_codes_length_file = encoder.load_data("./BL_Coding/" + path + "/BL_Codes_" + path + ".npy", "./BL_Coding/" + path + "/BL_Lengths_" + path + ".npy")
        
    # Calculate compression ratio
    compression = encoder.compression_ratio(integer_array, bl_codes_length)
    
    # Reconstruct original analogue values from integers and beamform to get the reconstructed image
    decoder = BL_decoder()
    decoded_integers = decoder.decode_data("./BL_Coding/" + path + "/BL_Codes_" + path + ".npy")
    
    # Beamform and plot the compressed image
    # Beamforming
    img_data = bf.beamform(decoded_integers, numba_active=True)

    _ = plot_image(np.abs(img_data), 
                   scatters_coords_xz=None,
                   elements_coords_xz=None,
                   framework='plotly',
                   title="BL_Coding_" + path,
                   image_x_range=image_x_range,
                   image_z_range=image_z_range,
                   db_range=db_range,
                   colorscale='Greys',
                   save_fig=True, 
                   show=True,
                   path_to_save='./BL_Coding/' + path)    
    
    # Prepare dataset to extract metrics of resolution and contrast

    # Read info from file
    f_d = h5py.File("../datasets/archive/archive_to_download/database/experiments/" + path + "/" + path + "_expe_dataset_" + typed + ".hdf5", "r")

    attributes = f_d.attrs

    US = f_d['US']
    attributes = US.attrs

    US_DATASET0000 = US['US_DATASET0000']
    attributes = US_DATASET0000.attrs

    type_s = attributes.__getitem__('type')

    attributes = US_DATASET0000.attrs

    keys_1 = list(f_d.keys());

    US = f_d['US']

    US_DATASET0000 = US['US_DATASET0000']

    data = US_DATASET0000['data']

    imag_n = data['imag']

    real_n = data['real'][()]

    PRF = US_DATASET0000['PRF']

    angles = US_DATASET0000['angles']

    initial_time = US_DATASET0000['initial_time']

    modulation_frequency = US_DATASET0000['modulation_frequency']

    probe_geometry = US_DATASET0000['probe_geometry']

    sampling_frequency = US_DATASET0000['sampling_frequency']

    sound_speed = US_DATASET0000['sound_speed']

    # Create dataset suitable for calculating metrics

    imag_np = np.zeros((75, 128, 3328))

    # Creation of dataset

    out_filename = './BL_Coding/bl_compressed_dataset.hdf5'
    comp_file = h5py.File(out_filename,'w')
    comp_file.close()
    comp_file = h5py.File(out_filename,'a')

    # Complete the structure

    comp_file.attrs.create('version', [b'v.0.0.40'], dtype='|S9')

    US_path = 'US'
    US_group = comp_file.require_group(US_path)

    US_dataset_group = US_group.require_group('US_DATASET0000')

    US_dataset_group.attrs.create('type', [b"US"], dtype='|S2')
    US_dataset_group.attrs.create('subtype', [b"CPW"], dtype='|S3')
    US_dataset_group.attrs.create('signal_format', [b"RF"], dtype='|S2')
    US_dataset_group.attrs.create('name', [b"CIRS 040GSE Wires"], dtype='|S18')
    US_dataset_group.attrs.create('version', [b"v1.96"], dtype='|S6')
    US_dataset_group.attrs.create('creation_date', [b"2016/03/9 17:25:22.38"], dtype='|S22')

    # angles
    US_dataset_group.create_dataset( 'angles', data=angles)

    # PRF
    US_dataset_group.create_dataset( 'PRF', data=PRF)

    # initial_time
    US_dataset_group.create_dataset( 'initial_time', data=initial_time)

    # modulation frequency
    US_dataset_group.create_dataset( 'modulation_frequency', data=modulation_frequency)

    # probe geometry
    US_dataset_group.create_dataset( 'probe_geometry', data=probe_geometry)

    # sampling frequency
    US_dataset_group.create_dataset( 'sampling_frequency', data=sampling_frequency)

    # sound speed
    US_dataset_group.create_dataset( 'sound_speed', data=sound_speed)

    # imag data
    data_group = US_dataset_group.require_group('data')
    data_group.create_dataset('imag', data=imag_np)

    # real data
    rf_range_data = decoded_integers/2047.5
    rf_data = rf_range_data - 1
    data_group.create_dataset('real', data=rf_data)

    comp_file.close()
    f_d.close()

    

In [ ]:
compression = encoder.compression_ratio(integer_array, bl_codes_length)

In [ ]:
# Encode data
path = 'resolution_distorsion'
bl_codes, bl_codes_length = encoder.encode_data("./BL_Coding/" + path + "/int_converted_values.npy")

# Save BL codes and their lengths to a file
encoder = BL_encoder()
encoder.save_data("./BL_Coding/", path + "/BL_Codes_" + path, bl_codes, path + "/BL_Lengths_" + path, bl_codes_length)

# Load BL codes and their lengths from a file
bl_codes_file, bl_codes_length_file = encoder.load_data("./BL_Coding/" + path + "/BL_Codes_" + path + ".npy", "./BL_Coding/" + path + "/BL_Lengths_" + path + ".npy")

In [ ]:
# Reconstruct original analogue values from integers and beamform to get the reconstructed image

decoder = BL_decoder()

decoded_integers = decoder.decode_data("./BL_Coding/resolution_distorsion/BL_Codes_resolution_distorsion.npy")
print(decoded_integers.shape)

In [ ]:
# Save decoded integers
decoder.save_decoded_int("./BL_Coding/resolution_distorsion/decoded_integers", decoded_integers)

In [ ]:
# Load decoded integers
decoded_integers_file = decoder.load_decoded_int("./BL_Coding/resolution_distorsion/decoded_integers.npy")
print(decoded_integers_file.shape)

In [ ]:
# Load BL codes and their lengths from a file
encoder = BL_encoder()
path = "resolution_distorsion"
bl_codes_file, bl_codes_length_file = encoder.load_data("./BL_Coding/" + path + "/BL_Codes_" + path + ".npy", "./BL_Coding/" + path + "/BL_Lengths_" + path + ".npy")

In [ ]:
# Run function

# 0 is resolution_distortion
# 1 is contrast_speckle

do_everything('RF',1)